##### The following libraries are needed to excute this code:
Keras;
Tensorflow CPU or GPU;
Matplotlib;
sklearn;
itertools

Download dataset: https://www.kaggle.com/chetankv/dogs-cats-images#dog%20vs%20cat.zip

# Using Convolutional Neural Networks to classify dogs and cats


### Imports

In [ ]:
# TensorFlow and keras (to build the neural network)
import tensorflow as tf
import keras

# Useful tools from keras to make code more legible (it is also possible to do this manually each time one requires a function) 
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *

# Tensorflow uses numpy arrays to process data
import numpy as np

# Useflul tool for visualisation
from matplotlib import pyplot as plt

# Useflul tool for evalutating the netwrok
from sklearn.metrics import confusion_matrix
import itertools

### Data aquirement and manipulation

In [ ]:
# Make sure to download the dataset beforehand and create 
# a blank python file named: "weigths.py" located in the dataset file
# File paths: These might differ on different devices
computer_path = '/Users/timourjavarmagnier/Downloads/dataset/'
train_path = computer_path + 'training_set'
test_path = computer_path + 'test_set'
weights_path = computer_path + 'weights.py'

In [ ]:
# Data generation from image
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224, 224), classes=['cats', 'dogs'], batch_size= 10)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224, 224), batch_size= 10)

In [ ]:
# Visualisation function
def plots(ims, figsize=(12, 6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if ims.shape[-1] != 3:
            ims = ims.transpose((0, 2, 3, 1))
    f = plt.figure(figsize=figsize)
    cols = len(ims) // rows if len(ims) % 2 == 0 else len(ims) // rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i + 1)
        sp.axis('off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
    plt.show()

In [ ]:
# Shows the training images along with their corresponding labels
imgs, labels = next(train_batches)
plots(imgs, titles=labels)

### Building the Network

In [ ]:
# Using the keras tool kits the following convolutional neural netwrok was built
model = keras.Sequential([
    
    # Convolutional layers: 
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), use_bias=True, padding='same'),
    
    # A ZeroPadding layer adds a layer of 0 around the image to prevent 
    # loss of information due to the convolutions. This is done by calling padding='same'
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), use_bias=True, padding='same'),
    
    # A maxpooling layer only recording the max pixel value of a 2 by 2 square to diminish the number of parameters
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), use_bias=True, padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), use_bias=True, padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),

    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), use_bias=True, padding='same'),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), use_bias=True, padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),

    # Transforms data into a single numpy array ready to be fed to the neural network
    Flatten(),

    # Hidden layers of the neural netwrok
    Dense(300, activation='relu'),
    
    # Dropouts temporarely deactivate neurons in training to prevent over-specialization
    Dropout(0.5),
    Dense(300, activation='relu'),
    Dropout(0.5),

    # Output layer
    Dense(2, activation='softmax')
])

In [ ]:
# Prints model
model.summary()

### Loading weights

In [ ]:
if weights_path:
    model.load_weights(weights_path)
    print("Weights have been loaded")

### Compile and train the model

In [ ]:
# Compiles the model with and Adam optimizer with a learning rate of 0.0001
# The loss fucntion is calculated as cateforical_croseentropy
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Trains the network on 10 epochs
model.fit_generator(train_batches, steps_per_epoch=4, epochs=1, verbose=2)

### Saving the trained weights

In [ ]:
model.save_weights(weights_path)

### Visualizing predictions

In [ ]:
predictions = model.predict_generator(test_batches, steps=2, verbose=2)

In [ ]:
print(predictions)